## Test on Properties

This notebook focuses on customizing parsers that can be used to extract information

In this case we are going to try to check for data patching and parsers at the same time. If the extracted properties misalign with compounds extracted, the project would be meaningless.


10/15/2019 meeting 

Challenge:

1. in parser, if a full name with a abbrev in () it will not recognize
2. too messy data paragraphs --> cannot efficiently recognize 


In [1]:
import logging
import re
import pandas as pd
import urllib
import time

import chemdataextractor as cde
from chemdataextractor import Document
from chemdataextractor.model import Compound, UvvisSpectrum, UvvisPeak, BaseModel, StringType, ListType, ModelType
from chemdataextractor.parse.common import hyphen,lbrct, dt, rbrct
from chemdataextractor.parse.base import BaseParser
from chemdataextractor.utils import first

from chemdataextractor.parse.actions import strip_stop
from chemdataextractor.parse.elements import W, I, T, R, Optional, ZeroOrMore, OneOrMore, Or, And
from chemdataextractor.parse.cem import chemical_name,cem, chemical_label, lenient_chemical_label, solvent_name
from chemdataextractor.doc import Paragraph, Sentence, Caption, Figure,Table, Heading
from chemdataextractor.doc.table import Table, Cell

In [2]:
from chemdataextractor.text.chem import SOLVENT_RE, INCHI_RE, SMILES_RE

In [3]:
# open and read files
f = open('../test_articles/paper0.pdf', 'rb')
doc = Document.from_file(f)
abstract = [11]
paras = doc.elements
cems = doc.cems

f1 = open('../test_articles/paper1.pdf', 'rb')
doc1 = Document.from_file(f1)
abstract1 = [7,8]
paras1 = doc1.elements
cems1 = doc1.cems

f2 = open('../test_articles/paper2.pdf', 'rb')
doc2 = Document.from_file(f2)
abstract2 = [7,8]
paras2 = doc2.elements
cems2 = doc2.cems

f3 = open('../test_articles/paper3.pdf', 'rb')
doc3 = Document.from_file(f3)
abstract3 = [10]
paras3 = doc3.elements
cems3 = doc3.cems

f4 = open('../test_articles/paper4.pdf', 'rb')
doc4 = Document.from_file(f4)
abstract4 = [12]
paras4 = doc4.elements
cems4 = doc4.cems

f5 = open('../test_articles/paper5.pdf', 'rb')
doc5 = Document.from_file(f5)
abstract5 = [3,4]
paras5 = doc5.elements
cems5 = doc5.cems

f6 = open('../test_articles/paper6.pdf', 'rb')
doc6 = Document.from_file(f6)
abstract6 = [5,6,7,8]
paras6 = doc6.elements
cems6 = doc6.cems

f7 = open('../test_articles/paper7.pdf', 'rb')
doc7 = Document.from_file(f7)
abstract7 = [11]
paras7 = doc7.elements
cems7 = doc7.cems

In [4]:
# split the paragraph into elements
paras = doc.elements
cems = doc.cems
a = doc.records.serialize()

PCE and FF works fine, as well as other quantities end in %. For other units, further customization required

Most of properties from literature have the same layout, so if one example works, the rest of them should work too.

Any unit with simple expression (1 component) is easy to extract. Otherwise a combination is needed.

An example from CDE. Tg parser's regex phrase is defined in the following code. Our parser should be based on this too.

In [5]:
# All hyphen and minus characters. Probably more robust than the hyph POS tag.
hyphen = R('^[\-‐‑⁃‒–—―−－⁻]$')

# All quote and apostrophe characters
quote = R('^[\'’՚Ꞌꞌ＇‘’‚‛"“”„‟`´’‘]$')

slash = W('/')

In [6]:
from chemdataextractor.model import Compound, UvvisSpectrum, UvvisPeak, BaseModel, StringType, ListType, ModelType
from chemdataextractor.parse.common import hyphen,lbrct, dt, rbrct
from chemdataextractor.parse.base import BaseParser
from chemdataextractor.utils import first
from chemdataextractor.parse.actions import strip_stop, merge, join
from chemdataextractor.parse.elements import W, I, T, R, Optional, ZeroOrMore, OneOrMore, Not, Any
from chemdataextractor.parse.cem import chemical_name
from chemdataextractor.doc import Paragraph, Sentence
from chemdataextractor.model import GlassTransition, Compound

log = logging.getLogger(__name__)

prefix = Optional(I('a')).hide() + (Optional(lbrct) + W('Tg') + Optional(rbrct) | I('glass') + Optional(I('transition')) + Optional((I('temperature') | I('range') | I('temp.'))) | W('transition') + Optional((I('temperature') | I('range') | I('temp.')))
                                    ).hide() + Optional(lbrct + W('Tg') + rbrct) + Optional(W('=') | I('of') | I('was') | I('is') | I('at')).hide() + Optional(I('in') + I('the') + I('range') + Optional(I('of')) | I('about') | ('around') | I('ca') | I('ca.')).hide()

delim = R('^[:;\.,]$')

# TODO: Consider allowing degree symbol to be optional. The prefix should be restrictive enough to stop false positives.
units = (W('°') + Optional(R('^[CFK]\.?$')) |
         W('K\.?'))('units').add_action(merge)

joined_range = R('^[\+\-–−]?\d+(\.\d+)?[\-–−~∼˜]\d+(\.\d+)?$')('value').add_action(merge)

spaced_range = (R('^[\+\-–−]?\d+(\.\d+)?$') + Optional(units).hide() + (R('^[\-–−~∼˜]$') +
                                                                        R('^[\+\-–−]?\d+(\.\d+)?$') | R('^[\+\-–−]\d+(\.\d+)?$')))('value').add_action(merge)

to_range = (R('^[\+\-–−]?\d+(\.\d+)?$') + Optional(units).hide() + (I('to') +
                                                                    R('^[\+\-–−]?\d+(\.\d+)?$') | R('^[\+\-–−]\d+(\.\d+)?$')))('value').add_action(join)

temp_range = (Optional(R('^[\-–−]$')) + (joined_range | spaced_range | to_range))('value').add_action(merge)

temp_value = (Optional(R('^[~∼˜\<\>]$')) + Optional(R('^[\-–−]$')) +
              R('^[\+\-–−]?\d+(\.\d+)?$'))('value').add_action(merge)

temp = Optional(lbrct).hide() + (temp_range | temp_value)('value') + Optional(rbrct).hide()

tg = (prefix + Optional(delim).hide() + temp + units)('tg')

bracket_any = lbrct + OneOrMore(Not(tg) + Not(rbrct) + Any()) + rbrct

cem_tg_phrase = (Optional(cem) + Optional(I('having')).hide() + Optional(delim).hide() + Optional(
    bracket_any).hide() + Optional(delim).hide() + Optional(lbrct) + tg + Optional(rbrct))('tg_phrase')

obtained_tg_phrase = ((cem | chemical_label) + (I('is') | I('are') | I('was')).hide() + (I('measured') |
                                                                                         I('obtained') | I('yielded')).hide() + ZeroOrMore(Not(tg) + Not(cem) + Any()).hide() + tg)('tg_phrase')

#tg_phrase = cem_tg_phrase | method1_phrase | method2_phrase | method3_phrase | obtained_tg_phrase
tg_phrase = cem_tg_phrase | obtained_tg_phrase


class TgParser(BaseParser):
    """"""
    root = tg_phrase

    #print ('outside parser', tg_phrase, type(tg_phrase))

    def interpret(self, result, start, end):
        compound = Compound(
            glass_transitions=[
                GlassTransition(
                    value=first(result.xpath('./tg/value/text()')),
                    units=first(result.xpath('./tg/units/text()'))
                )
            ]
        )
        cem_el = first(result.xpath('./cem'))
        if cem_el is not None:
            compound.names = cem_el.xpath('./name/text()')
            compound.labels = cem_el.xpath('./label/text()')
        yield compound

In [32]:
"""
DataExtractor.parsers.pce
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

This is built on top of the chemdataextractor module:
'chemdataextractor.parse.uvvis' as an example for implementaion of the
chemdataextractor pasing syntax.

Parses the power conversion efficiency, it can be applied to solar cells.
PCE is one key parameter to measure the performance of solar cells.

"""

import logging
import re

from chemdataextractor.model import Compound, UvvisSpectrum, UvvisPeak, BaseModel, StringType, ListType, ModelType
from chemdataextractor.parse.common import hyphen
from chemdataextractor.parse.base import BaseParser
from chemdataextractor.utils import first
from chemdataextractor.parse.actions import strip_stop
from chemdataextractor.parse.elements import W, I, T, R, Optional, ZeroOrMore, OneOrMore
from chemdataextractor.parse.cem import chemical_name
from chemdataextractor.doc import Paragraph, Sentence


class Pce(BaseModel):
    value = StringType()
    units = StringType()

Compound.pce_pattern = ListType(ModelType(Pce))

abbrv_prefix = (I(u'PCE') | I(u'PCEs') | I(u'pce')).hide()
words_pref = (I(u'power') + I(u'conversion') + I(u'efficiency')).hide()
hyphanated_pref = (I(u'power') + I(u'-') + I('conversion') + I(u'efficiency')).hide()
joined_range = R('^[\+\-–−]?\d+(\.\d+)?[\-–−~∼˜]\d+(\.\d+)?$')('value').add_action(merge)
spaced_range = (R('^[\+\-–−]?\d+(\.\d+)?$') + Optional(units).hide() + (R('^[\-–−~∼˜]$') +
                                                                        R('^[\+\-–−]?\d+(\.\d+)?$') | R('^[\+\-–−]\d+(\.\d+)?$')))('value').add_action(merge)
to_range = (R('^[\+\-–−]?\d+(\.\d+)?$') + Optional(units).hide() + (I('to') +
                                                                    R('^[\+\-–−]?\d+(\.\d+)?$') | R('^[\+\-–−]\d+(\.\d+)?$')))('value').add_action(join)

prefix = Optional(I('a')).hide() + (Optional(lbrct) + W('PCE') + Optional(rbrct) | I('power') + Optional(I('conversion')) + Optional((I('efficiency') | I('range'))) + Optional((I('temperature') | I('range')))
                                    ).hide() + Optional(lbrct + W('PCE') + rbrct) + Optional(W('=') | I('of') | I('was') | I('is') | I('at')).hide() + Optional(I('in') + I('the') + I('range') + Optional(I('of')) | I('about') | ('around') | I('%')).hide()

# prefix = abbrv_prefix | words_pref | hyphanated_pref

common_text = R('(\w+)?\D(\D+)+(\w+)?').hide()
units = (W(u'%') | I(u'percent'))(u'units')
value = R(u'\d+(\.\d+)?')(u'value')

pce_first  = (words_pref + (Optional(lbrct) + abbrv_prefix + Optional(rbrct)) + ZeroOrMore(common_text) + value + units)(u'pce')
# pce_first = (prefix + ZeroOrMore(common_text) + value + units)(u'pce')
pce_second = (prefix + value + units)(u'pce')
pce_pattern = pce_first | pce_second

class PceParser(BaseParser):
    root = pce_pattern

    def interpret(self, result, start, end):
        compound = Compound(
            pce_pattern=[
                Pce(
                    value=first(result.xpath('./value/text()')),
                    units=first(result.xpath('./units/text()'))
                )
            ]
        )
        yield compound

In [33]:
# Sentence.parsers.append(FfParser())
# Paragraph.parsers.append(FfParser())

# Sentence.parsers.append(VocParser())
# Paragraph.parsers.append(VocParser())

Sentence.parsers.append(PceParser())
Paragraph.parsers.append(PceParser())

In [34]:
doc = Document(
#     Heading('5,10,15,20-Tetra(4-carboxyphenyl)porphyrin (3).'),
#     Paragraph('a glass-transition temperature (Tg) of 20°C'),
    Paragraph('open circuit voltage (voc) equal to 0.7 V'),
    Paragraph('Voc of 0.8 V'),
    Paragraph('we have a PCE of 10%'),
    Paragraph('It has been found that PSHQ4 has a Tg of ca. 130°'),
    Paragraph('with the short circuit current density (Jsc) of 12 mAcm-2'),
    Paragraph('material with a fill factor (ff) of 0.2'),
)

rec = doc.records.serialize()

In [35]:
rec

[{'glass_transitions': [{'value': '130', 'units': '°'}]}]

In this case we can test on doc7 on data patching. 

In [12]:
doc7[11]

Paragraph(id=None, references=[], text='ABSTRACT: We report the synthesis, properties, and photo-\nvoltaic applications of new π-conjugated polymers having\nthiophene, 3,4-dihexylthiophene, and 1,3,4-oxadiazole (OXD) or\n1,3,4-thiadiazole (TD) units in the main chain, denoted as P1 and\nP2. They were synthesized by the Stille coupling reaction of 2,5-\nbis(trimethylstannyl)thiophene and the corresponding monomers\nof 2,5-bis(5′-bromo-3′,4′-dihexylthien-2′-yl)-1,3,4-oxadiazole or\n2,5-bis(5′-bromo-3′,4′-dihexylthien-2′-yl)-1,3,4-thiadiazole,\nre-\nspectively. The experimental results indicated that the introduc-\ntion of an electron-accepting moiety of OXD or TD lowered the\nhighest occupied molecular orbital (HOMO) energy levels,\nresulting in the higher the open-circuit voltage (Voc) values of\npolymer solar cells (PSCs). Indeed, the PSCs of P1 and P2\nshowed high Voc values in the range 0.80−0.90 V. The highest\nﬁeld-eﬀect transistor (FET) mobilities of P1 and P2 with the OXD and TD moieties, respectively, were 1.41 × 10−3 and 8.81 ×\n10−2 cm2 V−1 s−1. The higher mobility of P2 was related to its orderly nanoﬁbrillar structure, as evidenced from the TEM images.\nMoreover, the higher absorption coeﬃcient and smaller band gap of P2 provided a more eﬃcient light-harvesting ability. The\npower conversion eﬃciency (PCE) of the PSC based on P2:PCBM = 1:1 (w/w) reached 3.04% with a short-circuit current\ndensity (Jsc) value of 6.60 mA/cm2, a Voc value of 0.80 V, and a ﬁll factor (FF) value of 0.576 during the illumination of AM 1.5,\n100 mW/cm2. In comparison,\nthe electron-accepting moiety exhibited an inferior device\nperformance (FET mobility = 2.10 × 10−4 cm2 V−1 s−1 and PCE = 1.91%). The experimental results demonstrated that\nincorporating the electron-acceptor moiety into the polythiophene backbone could enhance the device performance due to the\nlow-lying HOMO levels, compact packing structure, and high charge carrier mobility. This is the ﬁrst report for the achievement\nof PCE > 3% using PSCs based on polythiophenes having TD units in the main chain.')

In [13]:
doc = Document(
    Heading('Abstract:'),
    Paragraph('We report the synthesis, properties, and photo- voltaic applications of new π-conjugated polymers having thiophene, 3,4-dihexylthiophene, and 1,3,4-oxadiazole (OXD) or 1,3,4-thiadiazole (TD) units in the main chain, denoted as P1 and P2. They were synthesized by the Stille coupling reaction of 2,5- bis(trimethylstannyl)thiophene and the corresponding monomers of 2,5-bis(5′-bromo-3′,4′-dihexylthien-2′-yl)-1,3,4-oxadiazole or 2,5-bis(5′-bromo-3′,4′-dihexylthien-2′-yl)-1,3,4-thiadiazole, re- spectively. '),
    Paragraph('The experimental results indicated that the introduc- tion of an electron-accepting moiety of OXD or TD lowered the highest occupied molecular orbital (HOMO) energy levels, resulting in the higher the open-circuit voltage (Voc) values of polymer solar cells (PSCs). Indeed, the PSCs of P1 and P2 showed high Voc values in the range 0.80−0.90 V. The highest ﬁeld-eﬀect transistor (FET) mobilities of P1 and P2 with the OXD and TD moieties, respectively, were 1.41 × 10−3 and 8.81 × 10−2 cm2 V−1 s−1. '),
    Paragraph('The higher mobility of P2 was related to its orderly nanoﬁbrillar structure, as evidenced from the TEM images. Moreover, the higher absorption coeﬃcient and smaller band gap of P2 provided a more eﬃcient light-harvesting ability. '),
    Paragraph('The power conversion eﬃciency (PCE) of the PSC based on P2:PCBM = 1:1 (w/w) reached 3.04 % with a short-circuit current density (Jsc) value of 6.60 mA/cm2, a Voc value of 0.80 V, and a fill factor value of 57.6% during the illumination of AM 1.5, 100 mW/cm2. '),
    Paragraph('In comparison, the electron-accepting moiety exhibited an inferior device performance (FET mobility = 2.10 × 10−4 cm2 V−1 s−1 and PCE = 1.91%). The experimental results demonstrated that incorporating the electron-acceptor moiety into the polythiophene backbone could enhance the device performance due to the low-lying HOMO levels, compact packing structure, and high charge carrier mobility. This is the ﬁrst report for the achievement of PCE > 3% using PSCs based on polythiophenes having TD units in the main chain.')
)

rec = doc.records.serialize()

### Example to run on the abstract

In [14]:
def abstract_ext(paras):
    """
    Abstract extraction from text
    [] paras
    """
    abstract = []
    for i in range(len(paras)):
        if ("Abstract" in paras[i].text or "ABSTRACT" in paras[i].text):
            abstract.append(paras[i])
    return abstract

In [15]:
abs_text = abstract_ext(paras)
abs_text1 = abstract_ext(paras1)
abs_text2 = abstract_ext(paras2)
abs_text3 = abstract_ext(paras3)
abs_text4 = abstract_ext(paras4)
abs_text5 = abstract_ext(paras5)
abs_text6 = abstract_ext(paras6)
abs_text7 = abstract_ext(paras7)

### Test on Abstracts

1. Test on Saeki sensei's paper
2. 

In [16]:
abstract_doc = doc.elements[11]
abstract_doc.records.serialize()

IndexError: list index out of range

In [ ]:
abstract1_doc = doc1.elements[7] + doc1.elements[8]
abstract1_doc.records.serialize()

In [ ]:
abstract2_doc = doc2.elements[7] + doc2.elements[8]
abstract2_doc.records.serialize()

In [ ]:
abstract3_doc = doc3.elements[10]
abstract3_doc.records.serialize()

In [ ]:
abstract4_doc = doc4.elements[12]
abstract4_doc.records.serialize()

In [ ]:
abstract5_doc = doc5.elements[3] + doc5.elements[4]
abstract5_doc.records.serialize()

In [ ]:
abstract6_doc = doc6.elements[5] + doc6.elements[6] + doc6.elements[7] + doc6.elements[8]
abstract6_doc.records.serialize()

In [ ]:
abstract7_doc = doc7.elements[11]
abstract7_doc.records.serialize()

### Refer to the tese dataset

In [ ]:
test = pd.read_csv("../test_articles/test.csv",sep="/t",delimiter="\\t")
test